### Creates a tSNE Visualization of the given PorphyStruct Result data

In [ ]:
import numpy as np
import pandas as pd
from data import constants
from mathutil import comp, percentage

In [ ]:
df = pd.read_excel(r"C:\Users\jenso\PowerFolders\Forschung\CSD Data Mining\Results.xlsx")
df.fillna("", inplace=True)
dataset = df.loc[(df["Type"] == "Corrole") & (df["Class"] == "Corrole")]
dataset.reset_index(drop=True, inplace=True)
dataset = percentage.createPercData(dataset)
dataset = comp.createCompData(dataset)
dataset["dominant"] = dataset[constants.compColumns].idxmax(axis=1)


In [ ]:
len(dataset)

### Prepare sklearn stuff

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import  SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

In [ ]:
preprocess = ColumnTransformer(
    transformers = [
        ("prepocess_ext", Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy="constant", fill_value=0)),
                ("scaler", StandardScaler())
            ]
        ), constants.percColumns)
    ], sparse_threshold=0)

### Do TSNE Computation

In [ ]:
tsne = TSNE(n_components=2, random_state=69420, perplexity=50, n_iter=5000)
df_mapping = pd.DataFrame(tsne.fit_transform(preprocess.fit_transform(dataset)), columns=["x", "y"])


### Add Columns for Viz

In [ ]:
df_mapping["M"] = dataset["Metal"]
df_mapping["Doop"] = dataset["DoopExp"]
df_mapping["Group"] = dataset["Group"].astype(str)
df_mapping["dominant"] = dataset["dominant"]
df_mapping["Cu"] = dataset["Metal"] == "Cu"
df_mapping["H"] = dataset["Metal"] == "H"
df_mapping["Class"] = dataset["Class"]



### Plot Stuff

In [ ]:
! pip install plotnine
! pip install natsort
! pip install -U kaleido
! pip install plotly

import plotly.express as px
from natsort import index_natsorted
import plotly.express as px
from plotnine import *
import plotly.io as pio
import plotly
plotly.__version__ #5.6.0
import kaleido #required
kaleido.__version__ #0.2.1

In [ ]:
df_mapping.sort_values(by="Group", key=lambda x: np.argsort(index_natsorted(df_mapping["Group"])), inplace=True)

In [ ]:
ggplot(df_mapping,aes(x="x", y="y", label="M")) + geom_point(aes(size="Doop", colour="Class")) + geom_text(size=6)

In [ ]:
metrix = ["Cu", "H", "dominant", "Group", "Class"]
for k in metrix:
    fig = px.scatter(df_mapping, x="x", y="y", color=k, size="Doop", text="M")
    fig.update_traces(textfont_size=8)
    pio.write_image(fig, f"out/tsne_{k}.png", scale=15)
